<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [3]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip


Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape


((59400, 39), (14358, 39), (59400, 2), (14358, 2))

In [0]:
from sklearn.cluster import KMeans

def add_local_gps(X3):
  points=X3[['longitude','latitude']]
  points = points.drop(points[points.longitude == 0].index)
  
  km = KMeans(n_clusters=20)
  km = km.fit(points)
  
  points['centroid']=km.labels_
  return points['centroid']

In [0]:
def num_cleanup(X1):
  '''
  date_recorded
  ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year']
  '''
  
  # categorical value nan converted to unknown to be processed by regression.
  X2 = X1.fillna('unknown')
  
  # added recorded_year_built
  # convert date_recorded as year 
  dates = pd.to_datetime(X2.date_recorded)
  X2['recorded_year_built'] = dates.dt.year - X2['construction_year']
  X2.loc[X2.recorded_year_built > 1000 ] = 0
  X2.date_recorded = dates.dt.year
  
  #
  bins=[-100,0,100,500,1000,1250,1500,2000]
  labels=[0,100,500,1000,1250,1500,2000]
  X2['gps_height_bin']= pd.cut(X2.gps_height, bins, labels=labels)
  #X2.drop(['gps_height'], axis=1, inplace=True)
  
  # replace longitude, latitude with centroid
  X2['centroid']=add_local_gps(X1)
  #X2.drop(['longitude','latitude'], axis=1, inplace=True)
  
  type_dict = {'amount_tsh':'float64',    # nunique()=98
               'date_recorded':'float64', # year
               'gps_height_bin':'float64',    #bins
               'centroid':'float64',  # replace longitude, latitude
               'num_private':'float64',   # nunique()=65
               'population':'float64',    # nunique()=1049
               'construction_year':'float64',   # nunique()=55
               'recorded_year_built':'float64', #added
               'region_code':'float64',   # nunique()=27
               'district_code':'float64'  # nunique=20 
              }
  X2 = X2.astype(dtype = type_dict)
  
  X2 = X2.fillna(0)

  return X2

In [0]:
from sklearn.cluster import KMeans

def viz_gps(X3):
  points=X3[['longitude','latitude']]
  points = points.drop(points[points.longitude == 0].index)

  '''
  sum_of_squared_distanced = []

  K = range(1,25)
  for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(points)
    sum_of_squared_distanced.append(km.inertia_)

  plt.plot(K, sum_of_squared_distanced, marker="X")
  plt.xlabel('k', fontsize=15)
  plt.ylabel('Sum_of_squared_distanced', fontsize=15)
  plt.title('Elbow Method to find the optimal k by eyeball', fontsize=15)
  plt.show()
'''
  km = KMeans(n_clusters=20)
  km = km.fit(points)

  plt.scatter(points.longitude, points.latitude, c=km.labels_, cmap='rainbow')

  plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], color='black')

In [0]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
df_train.select_dtypes(object).head(3)

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(categories='auto'),
    LogisticRegression(solver='lbfgs', multi_class='ovr',
                      max_iter=500))

In [0]:

df_clean = num_cleanup(df_train)

#include gps_height, longitude,latitude pump 0.63 to 0.76
df_num = df_clean.select_dtypes('number')
df_obj = df_clean.select_dtypes(object)


In [146]:
%%time
from sklearn.metrics import accuracy_score

X=df_num
y = df_label.status_group

pipe.fit(X,y)

y_pred = pipe.predict(X)
print('Accuracy score=', accuracy_score(y, y_pred))

Accuracy score= 0.7632154882154882
CPU times: user 45.9 s, sys: 28.1 s, total: 1min 14s
Wall time: 37.4 s


In [147]:
df_test_clean = num_cleanup(df_test)
df_test_num = df_test_clean.select_dtypes('number')
y_test_pred = pipe.predict(df_test_num)


ValueError: ignored

In [148]:
df_test_num.isnull().any()

amount_tsh             False
date_recorded          False
gps_height             False
longitude              False
latitude               False
num_private            False
region_code            False
district_code          False
population             False
construction_year      False
recorded_year_built    False
gps_height_bin         False
centroid               False
dtype: bool

In [139]:
df_train.amount_tsh.describe()

count     59400.000000
mean        303.833476
std        2866.486512
min           0.000000
25%           0.000000
50%           0.000000
75%          20.000000
max      350000.000000
Name: amount_tsh, dtype: float64